# From Black Box to Observable: Transforming Non-Compatible Agents to MLflow-Ready Systems

## Introduction

Building an AI agent is one thing—making it production-ready is another challenge entirely. The difference between a prototype agent and a production-grade system often comes down to one word: **observability**.

In this comprehensive guide, we'll dissect the exact changes needed to transform a basic LangGraph agent into an MLflow-compatible, production-ready system. We'll compare the two implementations side-by-side, explain why each change matters, and show you the real-world impact of these modifications.

By the end of this article, you'll understand not just *what* to change, but *why* these changes are critical for deploying reliable AI agents at scale.

## The Two Implementations: Overview

Let's start by understanding what we're comparing:

### Non-Compatible Agent
A straightforward LangGraph implementation that works perfectly in development but lacks the infrastructure needed for production deployment.

### MLflow-Compatible Agent
The same agent functionality, enhanced with MLflow's observability, configuration management, and deployment capabilities.

**The Promise**: Same behavior, better infrastructure.

## Side-by-Side Comparison

Let's examine each component and understand the differences.

---

## Change 1: State Definition

### Non-Compatible Version

```python
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
```

**What it does:**
- Defines a simple state structure
- Uses TypedDict for type hints
- Manages message history with add_messages reducer

**What's missing:**
- No MLflow metadata fields
- No tracing context
- No standardized format for deployment

### MLflow-Compatible Version

```python
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
```

**What changed:**
- Uses `ChatAgentState` from MLflow
- No need to define custom state class

**Why this matters:**

`ChatAgentState` is a pre-built state class that includes:

```python
# What ChatAgentState contains (conceptual):
class ChatAgentState(TypedDict):
    messages: Annotated[list, add_messages]  # Same as before
    # Plus MLflow-specific fields:
    trace_id: Optional[str]           # For linking spans
    run_id: Optional[str]             # For MLflow runs
    metadata: Optional[Dict]          # For custom tracking
    # And other internal fields for observability
```

**Real-World Impact:**

```python
# Non-Compatible: Just messages
state = {
    "messages": [HumanMessage(content="Hello")]
}

# MLflow-Compatible: Messages + Context
state = {
    "messages": [HumanMessage(content="Hello")],
    "trace_id": "tr_abc123",          # Automatic tracing
    "run_id": "run_xyz789",           # Experiment tracking
    "metadata": {                      # Custom metadata
        "user_id": "user_456",
        "session_id": "session_789"
    }
}
```

**Benefits:**
✅ Automatic trace linking across spans  
✅ Integration with MLflow experiments  
✅ Standardized format for deployment  
✅ Built-in metadata support  

---

## Change 2: Node Function Signature

### Non-Compatible Version

```python
def tool_calling_llm(state: State) -> State:
    """Execute tools based on tool calls in the last message"""
    current_state = state["messages"]
    return {"messages": [llm_with_tools.invoke(current_state)]}
```

**What it does:**
- Takes state as single parameter
- Extracts messages
- Invokes LLM
- Returns updated state

**What's missing:**
- No configuration parameter
- Can't pass runtime settings
- No tracing context propagation

### MLflow-Compatible Version

```python
def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
    """Execute tools based on tool calls in the last message"""
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**What changed:**
- Added `config: RunnableConfig` parameter
- Passes config to model invocation
- Uses model_runnable chain

**Why this matters:**

The `config` parameter is the key to everything:

```python
# Without config: Static behavior
def tool_calling_llm(state: State):
    # Temperature is hardcoded
    # Max tokens is fixed
    # No runtime customization
    response = llm.invoke(messages)
    return {"messages": [response]}

# With config: Dynamic behavior
def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
    # Temperature comes from config
    # Max tokens comes from config
    # Runtime customization enabled
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**Real-World Impact:**

```python
# Scenario: Same agent, different use cases

# Analytics request: Low temperature for precision
analytics_config = RunnableConfig(
    configurable={"temperature": 0.1, "max_tokens": 2000}
)
analytics_response = agent.invoke(state, config=analytics_config)

# Creative request: High temperature for creativity
creative_config = RunnableConfig(
    configurable={"temperature": 0.8, "max_tokens": 1500}
)
creative_response = agent.invoke(state, config=creative_config)

# Same agent, different behaviors!
```

**Benefits:**
✅ Runtime configuration changes  
✅ A/B testing without redeployment  
✅ Use-case specific optimization  
✅ Tracing context propagation  

---

## Change 3: Runnable Chain Construction

### Non-Compatible Version

```python
def tool_calling_llm(state: State) -> State:
    current_state = state["messages"]
    return {"messages": [llm_with_tools.invoke(current_state)]}
```

**What it does:**
- Directly invokes LLM
- Manually extracts messages
- Simple but not serializable

### MLflow-Compatible Version

```python
# Create preprocessing chain
preprocessor = RunnableLambda(lambda state: state["messages"])
model_runnable = preprocessor | model

def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**What changed:**
- Created explicit preprocessor step
- Chained preprocessor with model using pipe (`|`)
- Everything is now a Runnable

**Why this matters:**

The chain pattern creates a serializable pipeline:

```python
# Non-Compatible: Not serializable
def process(state):
    messages = state["messages"]  # Manual extraction
    return llm.invoke(messages)

# MLflow tries to serialize:
# ❌ Can't save "manual extraction" logic
# ❌ Can't track preprocessing step
# ❌ Can't deploy to Model Serving

# MLflow-Compatible: Fully serializable
preprocessor = RunnableLambda(lambda state: state["messages"])
model_runnable = preprocessor | model

# MLflow can serialize:
# ✅ Can save entire pipeline
# ✅ Can track each step
# ✅ Can deploy to Model Serving
```

**Execution Flow Comparison:**

```python
# Non-Compatible: Single-step
state → llm.invoke(state["messages"]) → response

# MLflow-Compatible: Multi-step pipeline
state → preprocessor → messages → model → response
         ↑              ↑         ↑        ↑
      Tracked     Logged    Traced  Monitored
```

**Real-World Impact:**

```python
# When you deploy to Databricks Model Serving:

# Non-Compatible:
# ❌ Deployment fails: "Cannot serialize function"
# ❌ No preprocessing visibility
# ❌ Can't inject config at deployment time

# MLflow-Compatible:
# ✅ Deployment succeeds
# ✅ Full pipeline visibility
# ✅ Runtime config injection works
# ✅ Each step is traced and monitored
```

**Benefits:**
✅ Complete serializability  
✅ Step-by-step tracing  
✅ Configuration injection  
✅ Deployment compatibility  

---

## Change 4: Tool Node Implementation

### Non-Compatible Version

```python
from langgraph.prebuilt.tool_node import ToolNode

builder.add_node("tools", ToolNode(tools=tools))
```

**What it does:**
- Uses standard LangGraph ToolNode
- Executes tools
- Returns results

**What's missing:**
- No automatic span creation
- No token tracking for tool calls
- No MLflow metadata

### MLflow-Compatible Version

```python
from mlflow.langchain.chat_agent_langgraph import ChatAgentToolNode

builder.add_node("tools", ChatAgentToolNode(tools=tools))
```

**What changed:**
- Uses `ChatAgentToolNode` instead of `ToolNode`
- Everything else stays the same

**Why this matters:**

`ChatAgentToolNode` is a wrapper around `ToolNode` that adds observability:

```python
# Standard ToolNode execution (conceptual):
class ToolNode:
    def invoke(self, state):
        tool_calls = extract_tool_calls(state)
        for tool_call in tool_calls:
            result = execute_tool(tool_call)  # Executed but not traced
        return results

# ChatAgentToolNode execution (conceptual):
class ChatAgentToolNode:
    def invoke(self, state):
        tool_calls = extract_tool_calls(state)
        for tool_call in tool_calls:
            with mlflow.start_span(name=tool_call.name, span_type="TOOL"):
                span.set_inputs(tool_call.args)
                result = execute_tool(tool_call)  # Executed AND traced
                span.set_outputs(result)
                span.set_attributes({"execution_time": duration})
        return results
```

**Real-World Impact:**

```python
# Non-Compatible ToolNode:
# Tool executes, but you don't know:
# ❌ Which tool was called
# ❌ What parameters were passed
# ❌ How long it took
# ❌ What results were returned
# ❌ If it succeeded or failed

# MLflow-Compatible ChatAgentToolNode:
# Full observability:
# ✅ Tool name logged
# ✅ Input parameters captured
# ✅ Execution time tracked
# ✅ Output results stored
# ✅ Success/failure status recorded

# Example trace:
{
    "span_id": "sp_004",
    "name": "search_web",
    "span_type": "TOOL",
    "inputs": {"query": "OpenAI latest news"},
    "outputs": {"results": "GPT-4 Turbo launched..."},
    "attributes": {
        "execution_time_ms": 3350,
        "tool_status": "success",
        "result_length": 487
    }
}
```

**Benefits:**
✅ Automatic tool execution tracing  
✅ Input/output capture  
✅ Performance monitoring  
✅ Error tracking  

---

## Change 5: Node Wrapping

### Non-Compatible Version

```python
builder.add_node("tool_calling_llm", tool_calling_llm)
```

**What it does:**
- Adds function directly to graph
- Function executes normally

**What's missing:**
- No automatic span creation
- Limited observability

### MLflow-Compatible Version

```python
builder.add_node("tool_calling_llm", RunnableLambda(tool_calling_llm))
```

**What changed:**
- Wrapped function with `RunnableLambda`
- Function becomes a Runnable

**Why this matters:**

`RunnableLambda` transforms regular functions into observable, serializable Runnables:

```python
# Non-Compatible: Plain function
def tool_calling_llm(state):
    return {"messages": [llm.invoke(state["messages"])]}

# Properties:
# - Not automatically traced
# - Not easily serializable
# - Can't inject config at runtime
# - No standardized interface

# MLflow-Compatible: Runnable-wrapped function
tool_calling_llm_runnable = RunnableLambda(tool_calling_llm)

# Properties:
# - Automatically traced
# - Fully serializable
# - Config injection works
# - Standardized Runnable interface
```

**Real-World Impact:**

```python
# When MLflow traces your agent:

# Non-Compatible:
# Graph execution: ✅ Traced
# Node functions: ❌ Not traced (black boxes)
# Result: Incomplete observability

# MLflow-Compatible:
# Graph execution: ✅ Traced
# Node functions: ✅ Traced (full visibility)
# Result: Complete observability

# Example trace hierarchy:
📊 Trace
├── StateGraph.invoke (CHAIN)
│   ├── tool_calling_llm (LLM) ← Visible because of RunnableLambda
│   ├── search_web (TOOL)      ← Visible because of ChatAgentToolNode
│   └── tool_calling_llm (LLM) ← Visible because of RunnableLambda
```

**Benefits:**
✅ Automatic span creation  
✅ Node-level tracing  
✅ Serialization support  
✅ Config injection capability  

---

## Change 6: Agent Class Structure

### Non-Compatible Version

```python
# Direct agent creation and usage
catalog = "agentic_ai"
schema = "databricks"
LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"

llm = ChatDatabricks(endpoint=LLM_ENDPOINT)
uc_tool_names = [f"{catalog}.{schema}.search_web"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools = [*uc_toolkit.tools]

agent = create_tool_calling_agent(llm, tools)

# Direct invocation
for event in agent.stream({"messages": user_input}, stream_mode='values'):
    event['messages'][-1].pretty_print()
```

**What it does:**
- Creates agent directly
- No wrapper class
- Simple but limited

**What's missing:**
- No MLflow ChatAgent interface
- Can't deploy to Model Serving
- No standardized predict method
- No configuration management

### MLflow-Compatible Version

```python
from mlflow.pyfunc import ChatAgent
from mlflow.models import ModelConfig
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

class DocsAgent(ChatAgent):
    def __init__(self, config, tools):
        """Initialize agent with configuration"""
        self.config = ModelConfig(development_config=config)
        self.tools = tools
        self.agent = self._build_agent_from_config()

    def _build_agent_from_config(self):
        """Build the agent graph with configured LLM"""
        llm = ChatDatabricks(
            endpoint=self.config.get("endpoint_name"),
            temperature=self.config.get("temperature"),
            max_tokens=self.config.get("max_tokens"),
        )
        agent = create_tool_calling_agent(llm, tools=self.tools)
        return agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        """Main prediction method - standardized interface"""
        request = {"messages": self._convert_messages_to_dict(messages)}
        output = self.agent.invoke(request)
        return ChatAgentResponse(**output)

# Configuration-based initialization
baseline_config = {
    "endpoint_name": LLM_ENDPOINT,
    "temperature": 0.01,
    "max_tokens": 1000
}

AGENT = DocsAgent(baseline_config, tools)

# Standardized invocation
result = AGENT.predict([{
    "role": "user",
    "content": "What is the latest news on OpenAI?"
}])
```

**What changed:**
- Created `ChatAgent` subclass
- Separated configuration from code
- Added standardized `predict()` method
- Implemented `ModelConfig` management

**Why this matters:**

The `ChatAgent` base class provides the deployment interface:

```python
# Non-Compatible: No standard interface
# Different ways to use:
agent.invoke(...)           # LangGraph way
agent.stream(...)           # Streaming way
agent.astream(...)          # Async way
# Problem: No consistency for deployment

# MLflow-Compatible: Standardized interface
# Always use:
result = agent.predict(messages, context)
# MLflow knows how to:
# - Log this agent
# - Deploy this agent
# - Monitor this agent
```

**Real-World Impact:**

```python
# Deployment to Databricks Model Serving:

# Non-Compatible:
# ❌ Can't deploy: No standard interface
# ❌ Can't log: No ChatAgent inheritance
# ❌ Can't monitor: No predict method

mlflow.pyfunc.log_model("agent", python_model=agent)
# Error: Agent must inherit from ChatAgent

# MLflow-Compatible:
# ✅ Can deploy: Standard ChatAgent interface
# ✅ Can log: Proper inheritance
# ✅ Can monitor: Standard predict method

mlflow.pyfunc.log_model(
    "agent",
    python_model=AGENT,
    model_config=baseline_config
)
# Success! Ready for deployment

# Once deployed, you can call it:
response = requests.post(
    "https://model-serving-endpoint",
    json={
        "messages": [{"role": "user", "content": "Hello"}]
    }
)
```

**Configuration Management Benefits:**

```python
# Non-Compatible: Hardcoded configuration
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.01,  # Hardcoded
    max_tokens=1000    # Hardcoded
)
# To change: Must modify code and redeploy

# MLflow-Compatible: External configuration
baseline_config = {
    "endpoint_name": "databricks-meta-llama-3-3-70b-instruct",
    "temperature": 0.01,
    "max_tokens": 1000
}
AGENT = DocsAgent(baseline_config, tools)

# To change: Just update config, no code changes
production_config = {
    "endpoint_name": "databricks-meta-llama-3-3-70b-instruct",
    "temperature": 0.1,   # Different setting
    "max_tokens": 2000    # Different setting
}
# Can override at deployment time without code changes!
```

**Benefits:**
✅ Deployment compatibility  
✅ Standardized interface  
✅ Configuration management  
✅ MLflow integration  

---

## Change 7: Message Format Handling

### Non-Compatible Version

```python
# Messages are used directly
user_input = "Explain AI in 2 bullets"
for event in agent.stream({"messages": user_input}, stream_mode='values'):
    event['messages'][-1].pretty_print()

# Output is printed directly
```

**What it does:**
- Simple string input
- Direct output consumption
- No format conversion

**What's missing:**
- No standardized message format
- Not compatible with MLflow's ChatAgentMessage
- Can't capture metadata

### MLflow-Compatible Version

```python
result = AGENT.predict([{
    "role": "user",
    "content": "What is the latest news on OpenAI product releases?"
}])

# result is ChatAgentResponse
print(result.messages[-1].content)
```

**What changed:**
- Messages follow standardized format
- Input: `ChatAgentMessage` format
- Output: `ChatAgentResponse` format

**Why this matters:**

Standardized formats enable monitoring and compatibility:

```python
# Non-Compatible: Various formats
"Hello"                          # String
["Hello"]                        # List
{"messages": "Hello"}            # Dict
HumanMessage(content="Hello")    # LangChain object
# Problem: Inconsistent, hard to monitor

# MLflow-Compatible: Standardized format
{
    "role": "user",              # Always has role
    "content": "Hello",          # Always has content
    "metadata": {...}            # Optional metadata
}
# Benefit: Consistent, easy to monitor
```

**Real-World Impact:**

```python
# When deployed to Model Serving:

# Non-Compatible:
# Request can be anything:
{"input": "hello"}
{"query": "hello"}
{"text": "hello"}
# Problem: No consistency, hard to integrate

# MLflow-Compatible:
# Request is always:
{
    "messages": [
        {"role": "user", "content": "hello"}
    ]
}
# Benefit: 
# - Frontend knows exact format
# - Monitoring tools understand structure
# - Easy integration with other systems
```

**Message Conversion Benefits:**

```python
class DocsAgent(ChatAgent):
    def predict(self, messages: list[ChatAgentMessage], ...) -> ChatAgentResponse:
        # Convert to internal format
        request = {"messages": self._convert_messages_to_dict(messages)}
        
        # Process
        output = self.agent.invoke(request)
        
        # Convert to standard output
        return ChatAgentResponse(**output)

# Benefits:
# ✅ Input validation
# ✅ Format normalization
# ✅ Metadata preservation
# ✅ Monitoring compatibility
```

**Benefits:**
✅ Standardized message format  
✅ Metadata support  
✅ Monitoring integration  
✅ API consistency  

---

## Complete Comparison Table

| Aspect | Non-Compatible | MLflow-Compatible | Impact |
|--------|----------------|-------------------|--------|
| **State** | Custom `TypedDict` | `ChatAgentState` | Automatic tracing context |
| **Node Signature** | `(state)` | `(state, config)` | Runtime configuration |
| **Chain** | Direct invocation | Runnable pipeline | Serializability |
| **Tool Node** | `ToolNode` | `ChatAgentToolNode` | Tool execution tracing |
| **Node Wrapper** | Direct function | `RunnableLambda` | Node-level tracing |
| **Agent Class** | No wrapper | `ChatAgent` subclass | Deployment interface |
| **Messages** | Various formats | `ChatAgentMessage` | Standardization |
| **Deployment** | ❌ Not possible | ✅ Ready | Production-ready |
| **Tracing** | ❌ Manual only | ✅ Automatic | Full observability |
| **Configuration** | ❌ Hardcoded | ✅ External | Runtime flexibility |

---

## Real-World Scenarios

### Scenario 1: Debugging in Production

**Non-Compatible Agent:**
```python
# User reports: "Agent is slow"
# You have: Application logs showing request/response
# You DON'T have:
# - Which step was slow?
# - Was it LLM or tool?
# - How many tokens used?
# - What were the intermediate outputs?

# Your options:
# 1. Add manual logging (requires code change + redeploy)
# 2. Guess based on timing (unreliable)
# 3. Try to reproduce locally (may not reproduce)
```

**MLflow-Compatible Agent:**
```python
# User reports: "Agent is slow"
# You have: Complete trace with all spans

trace = mlflow.get_trace(trace_id)

# Immediately see:
for span in trace.data.spans:
    duration = span.end_time_unix_ms - span.start_time_unix_ms
    print(f"{span.name}: {duration}ms")

# Output:
# tool_calling_llm: 2650ms
# search_web: 4500ms ← Bottleneck found!
# tool_calling_llm: 2100ms

# Solution: Optimize search_web tool
# No code changes needed to diagnose!
```

### Scenario 2: Cost Optimization

**Non-Compatible Agent:**
```python
# CFO asks: "How much are we spending on AI?"
# You have: API bills
# You DON'T have:
# - Token usage per request
# - Token usage per user
# - Token usage per use case
# - Which requests are expensive?

# Your options:
# 1. Estimate based on averages (inaccurate)
# 2. Add manual token tracking (code change)
# 3. Wait for next month's bill (too late)
```

**MLflow-Compatible Agent:**
```python
# CFO asks: "How much are we spending on AI?"
# You have: Complete token tracking

traces = mlflow.search_traces(
    experiment_names=["Production_Agent"],
    filter_string="timestamp > '2024-01-01'"
)

total_tokens = sum(
    t.info.token_usage['total_tokens'] 
    for t in traces 
    if t.info.token_usage
)

cost = total_tokens * 0.0001  # $0.10 per 1K tokens
print(f"Monthly cost: ${cost:.2f}")

# Breakdown by use case:
for use_case in ["analytics", "support", "creative"]:
    use_case_traces = mlflow.search_traces(
        filter_string=f"tags.use_case = '{use_case}'"
    )
    use_case_tokens = sum(
        t.info.token_usage['total_tokens']
        for t in use_case_traces
    )
    print(f"{use_case}: {use_case_tokens} tokens")
```

### Scenario 3: A/B Testing

**Non-Compatible Agent:**
```python
# Product Manager: "Let's test temperature 0.1 vs 0.5"
# Your process:
# 1. Create two separate agent instances
# 2. Deploy both
# 3. Manually route traffic
# 4. Manually collect metrics
# 5. Compare results manually

agent_a = create_agent(temperature=0.1)  # Deploy
agent_b = create_agent(temperature=0.5)  # Deploy again
# Complex setup, manual tracking
```

**MLflow-Compatible Agent:**
```python
# Product Manager: "Let's test temperature 0.1 vs 0.5"
# Your process:
# 1. Use same agent
# 2. Pass different config
# 3. MLflow tracks everything

agent = DocsAgent(config, tools)  # Deploy once

# Variant A
config_a = RunnableConfig(configurable={"temperature": 0.1})
mlflow.set_tags({"variant": "A"})
result_a = agent.invoke(state, config=config_a)

# Variant B
config_b = RunnableConfig(configurable={"temperature": 0.5})
mlflow.set_tags({"variant": "B"})
result_b = agent.invoke(state, config=config_b)

# Compare automatically:
traces_a = mlflow.search_traces(filter_string="tags.variant = 'A'")
traces_b = mlflow.search_traces(filter_string="tags.variant = 'B'")

# Metrics calculated automatically
```

---

## Migration Checklist

If you have a non-compatible agent and want to make it MLflow-compatible, follow this checklist:

### Step 1: Update State
```python
# Before
from typing_extensions import TypedDict
class State(TypedDict):
    messages: Annotated[list, add_messages]

# After
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
# Use ChatAgentState directly
```

### Step 2: Update Node Functions
```python
# Before
def tool_calling_llm(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

# After
def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

### Step 3: Create Runnable Chain
```python
# Before
# Direct invocation in function

# After
preprocessor = RunnableLambda(lambda state: state["messages"])
model_runnable = preprocessor | model
```

### Step 4: Update Tool Node
```python
# Before
from langgraph.prebuilt.tool_node import ToolNode
builder.add_node("tools", ToolNode(tools=tools))

# After
from mlflow.langchain.chat_agent_langgraph import ChatAgentToolNode
builder.add_node("tools", ChatAgentToolNode(tools=tools))
```

### Step 5: Wrap Node Functions
```python
# Before
builder.add_node("tool_calling_llm", tool_calling_llm)

# After
builder.add_node("tool_calling_llm", RunnableLambda(tool_calling_llm))
```

### Step 6: Create Agent Class
```python
# Before
agent = create_tool_calling_agent(llm, tools)

# After
class DocsAgent(ChatAgent):
    def __init__(self, config, tools):
        self.config = ModelConfig(development_config=config)
        self.tools = tools
        self.agent = self._build_agent_from_config()
    
    def _build_agent_from_config(self):
        llm = ChatDatabricks(
            endpoint=self.config.get("endpoint_name"),
            temperature=self.config.get("temperature"),
            max_tokens=self.config.get("max_tokens"),
        )
        return create_tool_calling_agent(llm, tools=self.tools)
    
    def predict(self, messages, context=None, custom_inputs=None):
        request = {"messages": self._convert_messages_to_dict(messages)}
        output = self.agent.invoke(request)
        return ChatAgentResponse(**output)

agent = DocsAgent(config, tools)
```

### Step 7: Standardize Messages
```python
# Before
result = agent.stream({"messages": "Hello"})

# After
result = agent.predict([{
    "role": "user",
    "content": "Hello"
}])
```

### Step 8: Enable Tracing
```python
# Add at the beginning
import mlflow
mlflow.langchain.autolog()
mlflow.set_experiment("My_Agent")
```

---

## Conclusion

The transformation from a non-compatible to MLflow-compatible agent isn't about changing functionality—it's about adding the infrastructure needed for production deployment:

### What Stays the Same
✅ Agent behavior and logic  
✅ Tool calling capabilities  
✅ Graph structure and flow  
✅ LLM integration  

### What Gets Better
✅ **Observability**: Complete trace visibility  
✅ **Configuration**: Runtime customization  
✅ **Deployment**: Model Serving ready  
✅ **Monitoring**: Automatic metrics tracking  
✅ **Debugging**: Pinpoint issues instantly  
✅ **Cost Tracking**: Token-level visibility  
✅ **Standardization**: Consistent interfaces  

### The Bottom Line

Non-compatible agents work great for prototypes and demos. But when you need to:
- Deploy to production
- Monitor performance
- Debug issues quickly
- Track costs accurately
- A/B test configurations
- Scale with confidence

...then MLflow-compatible agents are not just better—they're **essential**.

The changes might seem small, but the impact is transformational. You're not just building an agent anymore—you're building a **production-grade AI system** with the observability and control needed to succeed at scale.

---

## Quick Reference

**Minimum Changes for MLflow Compatibility:**

1. Use `ChatAgentState` instead of custom state
2. Add `config: RunnableConfig` to node functions
3. Create Runnable chain with preprocessor
4. Use `ChatAgentToolNode` instead of `ToolNode`
5. Wrap functions with `RunnableLambda`
6. Create `ChatAgent` subclass with `predict()` method
7. Standardize message formats
8. Enable `mlflow.langchain.autolog()`

**Result:** Production-ready agent with full observability!

Happy building! 🚀

In [0]:
'''''''